<a href="https://colab.research.google.com/github/yuvrajiro2outlook/firstgitrepo/blob/master/Random_Survival_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install lifelines

     |████████████████████████████████| 358kB 5.1MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-cp37-none-any.whl size=4050 sha256=6517ead64c41fbede34dd985b20309156fd4d2dec39d242f4363da048cbc31c0
  Stored in directory: /root/.cache/pip/wheels/dc/68/dc/91321c55fba449755524481854f5be70d41912b8f886f908bb
Successfully built autograd-gamma


In [70]:
import pandas as pd
import numpy as np
from lifelines import NelsonAalenFitter
from lifelines.statistics import logrank_test
from itertools import combinations
from joblib import Parallel, delayed


## Node

In [122]:
class Node:

    def __init__(self, x, y, tree, f_idxs, unique_deaths=3, min_leaf=3, timeline=None):
        """
        A Node of the Survival Tree.
        :param x: The input samples. Should be a Dataframe with the shape [n_samples, n_features].
        :param y: The target values as a Dataframe with the survival time in the first column and the event.
        :param tree: The corresponding Survival Tree
        :param f_idxs: The indices of the features to use.
        :param n_features: The number of features to use.
        :param unique_deaths: The minimum number of unique deaths required to be at a leaf node.
        :param min_leaf: The minimum number of samples required to be at a leaf node. A split point at any depth will
        only be considered if it leaves at least min_leaf training samples in each of the left and right branches.
        """
        self.x = x
        self.y = y
        self.tree = tree
        self.f_idxs = f_idxs
        self.unique_deaths = unique_deaths
        self.min_leaf = min_leaf
        self.score = 0
        self.split_val = None
        self.split_var = None
        self.lhs = None
        self.rhs = None
        self.chf = None
        self.chf_terminal = None
        self.terminal = False
        self.timeline = timeline
        self.grow_tree()

    def grow_tree(self):
        """
        Grow tree by calculating the Nodes recursively.
        :return: self
        """
        unique_deaths = np.sum(np.unique(self.y , axis = 0) , axis = 0)[1]

        if unique_deaths <= self.unique_deaths:
            self.compute_terminal_node()
            return self

        self.score, self.split_val, self.split_var, lhs_idxs_opt, rhs_idxs_opt = find_split(self)

        if self.split_var is None:
            self.compute_terminal_node()
            return self
        number_of_cols     = np.shape(self.x)[1]
        number_of_features = np.arange(np.int(np.ceil(np.sqrt(number_of_cols))))
        lf_idxs = np.random.permutation(number_of_cols)[number_of_features]
        rf_idxs = np.random.permutation(number_of_cols)[number_of_features]


        # Check Later whether lf_idxs or rf_idxs are necessary to pass since the
        # we are calculating feature index once again in this function

        self.lhs = Node(self.x[lhs_idxs_opt, :], self.y[lhs_idxs_opt, :], self.tree, lf_idxs,
                         min_leaf=self.min_leaf, timeline=self.timeline)

        self.rhs = Node(self.x[rhs_idxs_opt, :], self.y[rhs_idxs_opt, :], self.tree, rf_idxs,
                         min_leaf=self.min_leaf, timeline=self.timeline)

        return self

    def compute_terminal_node(self):
        """
        Compute the terminal node if condition has reached.
        :return: self
        """
        self.terminal = True
        self.chf = NelsonAalenFitter()
        t = self.y[:, 0]    # Time of The Event
        e = self.y[:, 1]    # Indicator for occurrence of event
        self.chf.fit(t, event_observed=e, timeline=self.timeline)
        return self

    def predict(self, x):
        """
        Predict the cumulative hazard function if its a terminal node. If not walk through the tree.
        :param x: The input sample.
        :return: Predicted cumulative hazard function if terminal node
        """

        # I am thinking of removing timeline argument from function compute_terminal_node
        # and change here cumulative_hazard_at_time with the timeline argument
        if self.terminal:
            self.tree.chf = self.chf.cumulative_hazard_.to_numpy()
            self.tree.chf = self.tree.chf[:, 0]
            return self.tree.chf

        else:
            if x[self.split_var] <= self.split_val:
                self.lhs.predict(x)
            else:
                self.rhs.predict(x)


## Functions for Splitting





In [123]:
def find_split(node):
    """
    Find the best split for a Node.
    :param node: Node to find best split for.
    :return: score of best split, value of best split, variable to split, left indices, right indices.
    """
    score_opt = -5000
    split_val_opt = None
    lhs_idxs_opt = None
    rhs_idxs_opt = None
    split_var_opt = None
    for i in node.f_idxs:
        #score, split_val, lhs_idxs, rhs_idxs = find_best_split_for_variable(node, i)
        score, split_val, lhs_idxs, rhs_idxs = find_best_split_for_variable(node.x[:,i], node.y , node.min_leaf)
        if score > score_opt:
            score_opt = score
            split_val_opt = split_val
            lhs_idxs_opt = lhs_idxs
            rhs_idxs_opt = rhs_idxs
            split_var_opt = i

    return score_opt, split_val_opt, split_var_opt, lhs_idxs_opt, rhs_idxs_opt

# def find_best_split_for_variable(node, var_idx):
def find_best_split_for_variable(x_var, y , min_leaf):
    """
    Find best split for a variable of a Node. Best split for a variable is the split with the highest log rank
    statistics. The logrank_test function of the lifelines package is used here.
    :param node: Node
    :param var_idx: Index of variable
    :return: score, split value, left indices, right indices.
    """
    # I am thiking of merging this and next function together (Later)
    #score, split_val, lhs_idxs, rhs_idxs = logrank_statistics(x=node.x, y=node.y,feature=var_idx,min_leaf=node.min_leaf)
    score, split_val, lhs_idxs, rhs_idxs = logrank_statistics(x_var, y,min_leaf)

    return score, split_val, lhs_idxs, rhs_idxs

# def logrank_statistics(x, y, feature, min_leaf):
def logrank_statistics(x_var, y, min_leaf):
    """
    Compute logrank_test of liflines package.
    :param x: Input samples
    :param y: Labels
    :param feature: Feature index
    :param min_leaf: Minimum number of leafs for each split.
    :return: best score, best split value, left indices, right indices
    """
    #x_feature = x[:, feature]
    score_opt = -5000
    split_val_opt = None
    lhs_idxs = None
    rhs_idxs = None
    thresholds = np.unique(np.sort(x_var))

    for split_val in thresholds:
        #feature1 = x_feature <= split_va
        feature1 = x_var <= split_val       #Creating an array of True , False which works as index
        feature2 = ~feature1
        if np.sum(feature1) < min_leaf or np.sum(feature2) < min_leaf:
            continue
        #durations_a = np.array(y[feature1, 0])
        #event_observed_a = np.array(y[feature1, 1])
        #durations_b = np.array(y[feature2, 0])
        #event_observed_b = np.array(y[feature2, 1])
        #score = logrank_test(durations_a, durations_b, event_observed_a, event_observed_b).test_statistic
        score = logrank_test(y[feature1, 0],y[feature2, 0] ,y[feature1, 1] , y[feature2, 1]).test_statistic
        if score > score_opt:
            score_opt = round(score, 3)
            split_val_opt = round(split_val, 3)
            lhs_idxs = feature1
            rhs_idxs = feature2

    return score_opt, split_val_opt, lhs_idxs, rhs_idxs


## Survival Tree

In [124]:
class SurvivalTree:

    #def __init__(self, x, y, f_idxs,  unique_deaths=3, min_leaf=3, timeline=None):
    def __init__(self, x, y,unique_deaths=3, min_leaf=3, timeline=None):
        """
        A Survival Tree to predict survival.
        :param x: The input samples. Should be a Dataframe with the shape [n_samples, n_features].
        :param y: The target values as a Dataframe with the survival time in the first column and the event.
        :param f_idxs: The indices of the features to use.
        :param n_features: The number of features to use.
        :param unique_deaths: The minimum number of unique deaths required to be at a leaf node.
        :param min_leaf: The minimum number of samples required to be at a leaf node. A split point at any depth will
        only be considered if it leaves at least min_leaf training samples in each of the left and right branches.
        """
        self.x = x
        self.y = y
        #self.f_idxs = f_idxs
        self.f_idxs = np.random.permutation(np.shape(x)[1])[np.arange(np.int(np.ceil(np.sqrt(np.shape(x)[1]))))]
        #self.n_features = n_features
        self.min_leaf = min_leaf
        self.unique_deaths = unique_deaths
        self.score = 0
        self.index = 0
        self.split_val = None
        self.split_var = None
        self.lhs = None
        self.rhs = None
        self.chf = None
        self.prediction_possible = None
        self.timeline = timeline
        self.grow_tree()

    def grow_tree(self):
        """
        Grow the survival tree recursively as nodes.
        :return: self
        """
        #unique_deaths = self.y.iloc[:, 1].reset_index().drop_duplicates().sum()[1]
        unique_deaths = np.sum(np.unique(self.y , axis = 0) , axis = 0)[1]

        self.score, self.split_val, self.split_var, lhs_idxs_opt, rhs_idxs_opt = find_split(self)

        if self.split_var is not None and unique_deaths > self.unique_deaths:
            self.prediction_possible = True
            number_of_cols     = np.shape(self.x)[1]
            number_of_features = np.arange(np.int(np.ceil(np.sqrt(number_of_cols))))
            lf_idxs = np.random.permutation(number_of_cols)[number_of_features]
            rf_idxs = np.random.permutation(number_of_cols)[number_of_features]

            self.lhs = Node(x=self.x[lhs_idxs_opt, :], y=self.y[lhs_idxs_opt, :],
                            tree=self, f_idxs=lf_idxs,
                            unique_deaths=self.unique_deaths, min_leaf=self.min_leaf,
                            timeline=self.timeline)

            self.rhs = Node(x=self.x[rhs_idxs_opt, :], y=self.y[rhs_idxs_opt, :],
                            tree=self, f_idxs=rf_idxs,
                            unique_deaths=self.unique_deaths, min_leaf=self.min_leaf,
                            timeline=self.timeline)

            return self
        else:
            self.prediction_possible = False
            return self

    def predict(self, x):
        """
        Predict survival for x.
        :param x: The input sample.
        :return: The predicted cumulative hazard function.
        """
        if x[self.split_var] <= self.split_val:
            self.lhs.predict(x)
        else:
            self.rhs.predict(x)
        return self.chf


## Random Survival Forest

In [125]:
class RandomSurvivalForest:

    def __init__(self, n_estimators=100, min_leaf=3, unique_deaths=3,
                 n_jobs=None, parallelization_backend="multiprocessing", oob_score=False):
        """
        A Random Survival Forest is a prediction model especially designed for survival analysis.
        :param n_estimators: The numbers of trees in the forest.
        :param min_leaf: The minimum number of samples required to be at a leaf node. A split point at any depth will
        only be considered if it leaves at least min_leaf training samples in each of the left and right branches.
        :param unique_deaths: The minimum number of unique deaths required to be at a leaf node.
        :param n_jobs: The number of jobs to run in parallel for fit. None means 1.
        """
        self.n_estimators = n_estimators
        self.min_leaf = min_leaf
        self.unique_deaths = unique_deaths
        self.n_jobs = n_jobs
        self.parallelization_backend = parallelization_backend
        self.bootstrap_idxs = None
        self.bootstraps = []
        self.oob_idxs = None
        self.oob_score = oob_score
        self.trees = []
        self.timeline = None

    def fit(self, x, y):
        """
        Build a forest of trees from the training set (X, y).
        :param x: The input samples. Should be a Dataframe with the shape [n_samples, n_features].
        :param y: The target values as a Dataframe with the survival time in the first column and the event
        in the second with the shape [n_samples, 2]
        :return: self: object
        """
        # Check The Timeline Argument Once Again
        self.timeline =  np.sort(y[:,0])  #np.arange(y[:, 0].min(), y[:, 0].max(), 1)
        if self.n_jobs == -1:
            self.n_jobs = multiprocessing.cpu_count()
        elif self.n_jobs is None:
            self.n_jobs = 1
        self.bootstrap_idxs = self.draw_bootstrap_samples(x)

        trees = Parallel(n_jobs=self.n_jobs, backend=self.parallelization_backend)(delayed(self.create_tree)(x, y, i)
                                                                                   for i in range(self.n_estimators))

        for i in range(len(trees)):
            if trees[i].prediction_possible:
                self.trees.append(trees[i])
                self.bootstraps.append(self.bootstrap_idxs[i])

        if self.oob_score:
            self.oob_score = self.compute_oob_score(x, y)

        return self

    def create_tree(self, x, y, i):
        """
        Grows a survival tree for the bootstrap samples.
        :param y: label data frame y with survival time as the first column and event as second
        :param x: feature data frame x
        :param i: Indices
        :return: SurvivalTree
        """
        #n_features = int(round(np.sqrt(x.shape[1]), 0))
        #f_idxs = np.random.permutation(x.shape[1])[:n_features]
        #number_of_cols = np.shape(x)[1]
        #number_of_features = np.arange(np.int(np.ceil(np.sqrt(number_of_cols))))
        #f_idxs = np.random.permutation(number_of_cols)[number_of_features]

        tree = SurvivalTree(x=x[self.bootstrap_idxs[i], :], y=y[self.bootstrap_idxs[i], :],
                                unique_deaths=self.unique_deaths, min_leaf=self.min_leaf,
                                timeline=self.timeline)

        return tree

    def compute_oob_ensembles(self, xs):
        """
        Compute OOB ensembles.
        :return: List of oob ensemble for each sample.
        """
        results = [compute_oob_ensemble_chf(sample_idx=sample_idx, xs=xs, trees=self.trees,
                                            bootstraps=self.bootstraps) for sample_idx in range(xs.shape[0])]
        oob_ensemble_chfs = [i for i in results if not (i.size ==0)]
        return oob_ensemble_chfs

    def compute_oob_score(self, x, y):
        """
        Compute the oob score (concordance-index).
        :return: c-index of oob samples
        """
        oob_ensembles = self.compute_oob_ensembles(x)
        c = concordance_index(y_time=y[:, 0], y_pred=oob_ensembles, y_event=y[:, 1])
        return c

    def predict(self, xs):
        """
        Predict survival for xs.
        :param xs: The input samples
        :return: List of the predicted cumulative hazard functions.
        """
        ensemble_chfs = [compute_ensemble_chf(sample_idx=sample_idx, xs=xs, trees=self.trees)
                         for sample_idx in range(xs.shape[0])]
        return ensemble_chfs

    def draw_bootstrap_samples(self, data):
        """
        Draw bootstrap samples
        :param data: Data to draw bootstrap samples of.
        :return: Bootstrap indices for each of the trees
        """
        bootstrap_idxs = []
        for i in range(self.n_estimators):
            no_samples = len(data)
            data_rows = range(no_samples)
            bootstrap_idx = np.random.choice(data_rows, no_samples)
            bootstrap_idxs.append(bootstrap_idx)

        return bootstrap_idxs


def compute_ensemble_chf(sample_idx, xs, trees):
    denominator = 0
    numerator = 0
    for b in range(len(trees)):
        sample = xs[sample_idx]
        chf = trees[b].predict(sample)
        denominator = denominator + 1
        numerator = numerator + 1 * chf
    ensemble_chf = numerator / denominator
    return ensemble_chf


def compute_oob_ensemble_chf(sample_idx, xs, trees, bootstraps):
    denominator = 0
    numerator = 0
    for b in range(len(trees)):
        if sample_idx not in bootstraps[b]:
            sample = xs[sample_idx]
            chf = trees[b].predict(sample)
            denominator = denominator + 1
            numerator = numerator + 1 * chf
    if denominator != 0:
        oob_ensemble_chf = numerator / denominator
    else:
        oob_ensemble_chf = pd.Series()
    return oob_ensemble_chf


## Scoring

In [126]:
def concordance_index(y_time, y_pred, y_event):
    """
    Compute concordance index.
    :param y_time: Actual Survival Times.
    :param y_pred: Predicted cumulative hazard functions.
    :param y_event: Actual Survival Events.
    :return: c-index.
    """
    predicted_outcome = [x.sum() for x in y_pred]
    possible_pairs = (combinations(range(len(y_pred)), 2))
    concordance = 0
    permissible = 0
    for i,j in possible_pairs:
        t1 = y_time[i]
        t2 = y_time[j]
        e1 = y_event[i]
        e2 = y_event[j]
        predicted_outcome_1 = predicted_outcome[i]
        predicted_outcome_2 = predicted_outcome[j]

        shorter_survival_time_censored = (t1 < t2 and e1 == 0) or (t2 < t1 and e2 == 0)
        t1_equals_t2_and_no_death = (t1 == t2 and (e1 == 0 and e2 == 0))

        if shorter_survival_time_censored or t1_equals_t2_and_no_death:
            continue
        else:
            permissible = permissible + 1
            if t1 != t2:
                if t1 < t2:
                    if predicted_outcome_1 > predicted_outcome_2:
                        concordance = concordance + 1
                        continue
                    elif predicted_outcome_1 == predicted_outcome_2:
                        concordance = concordance + 0.5
                        continue
                elif t2 < t1:
                    if predicted_outcome_2 > predicted_outcome_1:
                        concordance = concordance + 1
                        continue
                    elif predicted_outcome_2 == predicted_outcome_1:
                        concordance = concordance + 0.5
                        continue
            elif t1 == t2:
                if e1 == 1 and e2 == 1:
                    if predicted_outcome_1 == predicted_outcome_2:
                        concordance = concordance + 1
                        continue
                    else:
                        concordance = concordance + 0.5
                        continue
                elif not (e1 == 1 and e2 == 1):
                    if e1 == 1 and predicted_outcome_1 > predicted_outcome_2:
                        concordance = concordance + 1
                        continue
                    elif e2 == 1 and predicted_outcome_2 > predicted_outcome_1:
                        concordance = concordance + 1
                        continue
                    else:
                        concordance = concordance + 0.5
                        continue

    c = concordance / permissible

    return c


In [127]:
veteran = pd.read_csv("veteran (1).csv")

In [128]:
y = veteran.loc[:, ["time","status"]].to_numpy()
X = veteran.drop(["time","status"], axis=1).to_numpy()

In [129]:
rsf = RandomSurvivalForest(n_estimators=10,min_leaf = 15,oob_score = True)

In [133]:
rsf.fit(X, y)


In [134]:
rsf.oob_score

0.6700214859210675

In [85]:
np.int(5.4)

5

In [66]:
class M:
  def __init__(self):
    self.a = 5
    self.set_b()

  def set_b(self):
    self.b = 6

In [67]:
a = M()

In [68]:
a.b

6